<a href="https://colab.research.google.com/github/kamalsinha7305/Pneumonia-Detection-in-Chest-X-Ray-Images-Using-CNN-and-Explainable-AI/blob/main/pnemonia_detection_in_check_xray_FINAL_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Part 1: Imports and Setup
# Standard imports from your original notebook
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras import layers, models

# New import for Transfer Learning
import tensorflow_hub as hub

# Scikit-learn for evaluation
from sklearn.metrics import classification_report, confusion_matrix

# OpenCV for Grad-CAM visualization
import cv2

print("TensorFlow Version:", tf.__version__)
print("TensorFlow Hub Version:", hub.__version__)

In [ ]:
# Part 1: Imports and Data Download (Corrected)

import kagglehub
# This command downloads the dataset and returns the path where it's stored.
# This MUST be the first step to ensure the files exist.
print("Downloading dataset from Kaggle Hub...")
dataset_path_str = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

# Now, create the Path object from the returned string
BASE = Path(dataset_path_str)

print(f"Dataset downloaded to: {BASE}")
print(f"Directory exists: {BASE.exists()}")
print(f"Contents of BASE: {os.listdir(BASE)}")


# Part 2: Dataset Loading and Preparation (This should now work)
IMG_SIZE = (224, 224)
BATCH = 32

# The actual data is in a subfolder named 'chest_xray'
DATA_DIR = BASE / 'chest_xray'

# Part 2: Dataset Loading and Preparation (with Augmentation in the Pipeline)

# Define the data augmentation as a sequential model
# This keeps it separate from the main model architecture
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
], name="data_augmentation")


def make_ds(folder, shuffle=True, augment=False):
    """
    Loads a dataset, normalizes it, and optionally applies data augmentation.
    """
    ds = tf.keras.utils.image_dataset_from_directory(
        folder,
        image_size=IMG_SIZE,
        batch_size=BATCH,
        label_mode="binary",
        shuffle=shuffle
    )
    class_names = ds.class_names

    # Normalize the images to [0, 1]
    ds = ds.map(lambda x, y: (x / 255.0, y), num_parallel_calls=tf.data.AUTOTUNE)

    # Apply augmentation ONLY to the training dataset
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)

    # Prefetch for performance
    return ds.prefetch(tf.data.AUTOTUNE), class_names

# Load the datasets using the new function
train_ds, class_names = make_ds(DATA_DIR / "train", shuffle=True, augment=True)
val_ds, _ = make_ds(DATA_DIR / "val", shuffle=False, augment=False)
test_ds, _ = make_ds(DATA_DIR / "test", shuffle=False, augment=False)

print("\nSuccessfully loaded datasets with augmentation in the pipeline!")
print("Classes:", class_names)

In [ ]:
# Part 1 and 2 remain the same (Imports and Data Loading)
# ... Ensure your data (train_ds, val_ds, test_ds) is loaded before running this ...

# Import the necessary components from Keras applications
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D

# Part 3: Advanced Model Building - FINAL FIX v3
# Part 3: Advanced Model Building (Simplified and Corrected)
# Part 3: Advanced Model Building (Simplified and Corrected)

def build_stable_model(input_shape=(224, 224, 3)):
    """
    Builds the model WITHOUT internal data augmentation layers.
    """
    # 1. Load the base model
    base_model = MobileNetV2(input_shape=input_shape,
                             include_top=False,
                             weights='imagenet')
    base_model.trainable = False # Freeze the base

    # 2. Build the new model using the Functional API
    inputs = layers.Input(shape=input_shape)

    # 3. The input is passed directly to the base model
    x = base_model(inputs, training=False)

    # 4. Add the classifier head
    x = GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    # 5. Create and compile the final model
    model = models.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Build the new, stable model
model = build_stable_model()
model.summary()




In [ ]:
# Part 4: Model Training
# We use the same callbacks as before for robust training.
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor='val_accuracy'),
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2, verbose=1)
]
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# --- 5. ENHANCED PERFORMANCE EVALUATION ---
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve

# --- Original Evaluation Code ---
# Get true labels and predictions from the test set
y_true = np.concatenate([y.numpy().ravel() for _, y in test_ds])
y_prob = model.predict(test_ds).ravel() # Probabilities from the model
y_pred = (y_prob >= 0.5).astype(int) # Convert probabilities to class labels (0 or 1)

# Print Classification Report
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_names))

# Plot Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(cm, cmap='Blues')
ax.set_xticks([0, 1]); ax.set_yticks([0, 1])
ax.set_xticklabels(class_names); ax.set_yticklabels(class_names)
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.title('Confusion Matrix', fontsize=14)
for (i, j), v in np.ndenumerate(cm):
    ax.text(j, i, str(v), ha='center', va='center', color='white' if cm[i, j] > cm.max()/2 else 'black', fontsize=14)
plt.show()

# --- ADDITION 1: ROC CURVE AND AUC SCORE ---
fpr, tpr, thresholds = roc_curve(y_true, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

print(f"\nArea Under Curve (AUC) Score: {roc_auc:.4f}")

# --- ADDITION 2: VISUALIZE MISCLASSIFIED IMAGES ---
# We need an unprocessed dataset for clean visualization
unprocessed_test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, "test"), image_size=IMG_SIZE, batch_size=BATCH, shuffle=False
)

# Find indices of misclassified images
misclassified_indices = np.where(y_pred != y_true)[0]
correct_indices = np.where(y_pred == y_true)[0]

# Unbatch the dataset to access individual images
all_images = np.concatenate([x for x, y in unprocessed_test_ds])
all_labels = np.concatenate([y for x, y in unprocessed_test_ds])

# Plot a few False Positives (Normal predicted as Pneumonia)
print("\n--- Examples of Misclassified Images ---\n")
fp_indices = np.intersect1d(np.where(y_true == 0), np.where(y_pred == 1))
print("False Positives (NORMAL predicted as PNEUMONIA):")
plt.figure(figsize=(10, 4))
for i, index in enumerate(fp_indices[:5]): # Show up to 5 examples
    plt.subplot(1, 5, i + 1)
    plt.imshow(all_images[index].astype('uint8'))
    plt.title(f"Pred: PNEUMONIA\nTrue: NORMAL\nProb: {y_prob[index]:.2f}")
    plt.axis('off')
plt.show()

# Plot a few False Negatives (Pneumonia predicted as Normal)
fn_indices = np.intersect1d(np.where(y_true == 1), np.where(y_pred == 0))
print("\nFalse Negatives (PNEUMONIA predicted as NORMAL):")
plt.figure(figsize=(10, 4))
for i, index in enumerate(fn_indices[:5]): # Show up to 5 examples
    plt.subplot(1, 5, i + 1)
    plt.imshow(all_images[index].astype('uint8'))
    plt.title(f"Pred: NORMAL\nTrue: PNEUMONIA\nProb: {y_prob[index]:.2f}")
    plt.axis('off')
plt.show()

In [ ]:
!pip install lime

In [ ]:
# Part 6 (Alternative 1): LIME (Local Interpretable Model-agnostic Explanations)
# Make sure these imports are at the TOP of your notebook
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import applications # <-- THE FIX IS ADDING THIS LINE

print("\n--- Generating LIME Explainability Visualization ---")

# We need an unprocessed dataset for clean visualization
unprocessed_test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, "test"),
    image_size=IMG_SIZE,
    batch_size=1,
    shuffle=True
)

# Function to make predictions in the format LIME expects
def predict_for_lime(images):
    # LIME provides images in a different format, so we preprocess them for VGG16
    processed_images = applications.vgg16.preprocess_input(images)
    # Get the model's prediction probability for the "PNEUMONIA" class
    preds = model.predict(processed_images)
    # We need to return probabilities for both classes: [P(Normal), P(Pneumonia)]
    return np.hstack((1 - preds, preds))

# Create a LIME explainer object
explainer = lime_image.LimeImageExplainer()

# Take one image from the test set to explain
for img_batch, label_batch in unprocessed_test_ds.take(1):
    original_img = img_batch[0].numpy().astype('uint8')
    true_label_index = int(label_batch.numpy()[0])

    print("Generating LIME explanation... (This may take a moment)")

    # Generate the explanation
    explanation = explainer.explain_instance(
        original_img,
        predict_for_lime,
        top_labels=2,
        hide_color=0,
        num_samples=1000
    )

    # Get the image and mask for the top predicted label
    temp, mask = explanation.get_image_and_mask(
        explanation.top_labels[0],
        positive_only=True,
        num_features=5,
        hide_rest=True
    )

    # Get the model's prediction
    pred_prob = model.predict(applications.vgg16.preprocess_input(img_batch))[0][0]
    prediction = "PNEUMONIA" if pred_prob > 0.5 else "NORMAL"
    true_label = class_names[true_label_index]

    # Display the result
    plt.figure(figsize=(10, 5))
    ax1 = plt.subplot(1, 2, 1)
    ax1.imshow(original_img)
    ax1.set_title(f"Original Image\nTrue Label: {true_label}")
    ax1.axis('off')

    ax2 = plt.subplot(1, 2, 2)
    ax2.imshow(mark_boundaries(temp / 2 + 0.5, mask))
    ax2.set_title(f"LIME Explanation\nPrediction: {prediction}")
    ax2.axis('off')

    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Part 7: In-Text Explanation Generator ---

def generate_text_explanation(pred_prob, true_label_index, class_names):
    """
    Generates a human-readable text report based on the model's prediction.
    """
    prediction_index = 1 if pred_prob > 0.5 else 0
    prediction_name = class_names[prediction_index]
    true_label_name = class_names[true_label_index]

    report = f"--- AI Analysis Report ---\n"
    report += f"True Label: {true_label_name}\n\n"

    confidence_str = ""
    evidence_str = ""

    if prediction_name == 'PNEUMONIA':
        confidence = pred_prob * 100
        if confidence > 90:
            confidence_str = f"Finding: PNEUMONIA (High Confidence: {confidence:.1f}%)"
            evidence_str = "Evidence: Strong indicators of pneumonia, such as significant lung opacities, were detected."
        elif confidence > 70:
            confidence_str = f"Finding: PNEUMONIA (Moderate Confidence: {confidence:.1f}%)"
            evidence_str = "Evidence: Features consistent with pneumonia were detected. Clinical correlation is advised."
        else:
            confidence_str = f"Finding: PNEUMONIA (Low Confidence: {confidence:.1f}%)"
            evidence_str = "Evidence: Some indicators suggestive of pneumonia were found, but the signs are not definitive."

    else: # NORMAL
        confidence = (1 - pred_prob) * 100
        if confidence > 90:
            confidence_str = f"Finding: NORMAL (High Confidence: {confidence:.1f}%)"
            evidence_str = "Evidence: No significant indicators of pneumonia were detected in the visible lung fields."
        else:
            confidence_str = f"Finding: NORMAL (Low Confidence: {confidence:.1f}%)"
            evidence_str = "Evidence: The scan appears largely normal, but some minor ambiguities prevent a high-confidence assessment."

    report += f"{confidence_str}\n"
    report += f"{evidence_str}\n"
    report += "Recommendation: For clinical use, this analysis requires review by a qualified radiologist.\n"

    return report

# --- Generate and Print Reports for a Few Test Images ---

# Get the unprocessed images and their true labels again
unprocessed_test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, "test"),
    image_size=IMG_SIZE,
    batch_size=1,
    shuffle=True
)

print("\n--- Generating In-Text Explanations for Test Images ---\n")

for img_batch, label_batch in unprocessed_test_ds.take(3): # Generate for 3 examples
    # Preprocess the image to get a prediction
    processed_img = applications.vgg16.preprocess_input(img_batch)
    prediction_probability = model.predict(processed_img)[0][0]

    # Generate the report
    true_label_idx = int(label_batch.numpy()[0])
    explanation_report = generate_text_explanation(prediction_probability, true_label_idx, class_names)

    # Display the image and its report
    plt.imshow(img_batch[0].numpy().astype('uint8'))
    plt.axis('off')
    plt.show()
    print(explanation_report)
    print("-" * 30)
